In [1]:
import polars as pl
from src.tank_ts_endpoint import fetch_tank_ts_data, GetTanksTimeDataReq

#pl.Config.set_tbl_rows(2000)

req = GetTanksTimeDataReq(source_key=['9883901', '98839FAC'])

df = await fetch_tank_ts_data(req)
if df is None:
    raise ValueError("df is None")
lf = df.lazy()
lf.collect()

ts,value,source_key,metric_nice_name,uom
datetime[μs],f64,str,str,str
2024-08-22 20:10:28.260583,291.456951,"""9883901""","""OilTank1Volume""","""bbl"""
2024-08-26 18:34:13.231329,298.883254,"""9883901""","""OilTank1Volume""","""bbl"""
2024-08-28 06:15:32.821494,262.238814,"""9883901""","""OilTank1Volume""","""bbl"""
2024-08-26 09:32:03.681955,303.781195,"""9883901""","""OilTank1Volume""","""bbl"""
2024-08-29 10:53:08.232747,351.712976,"""9883901""","""OilTank1Volume""","""bbl"""
…,…,…,…,…
2024-08-28 22:23:33.615018,232.012233,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in"""
2024-08-28 13:59:08.358302,198.544071,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in"""
2024-08-29 07:27:55.213780,272.102104,"""98839FAC""","""ESD-OilTankInchesUntilAlarm""","""in"""


In [2]:
import polars as pl

# Define custom orders
tank_metrics = ["Level", "Volume", "InchesUntilAlarm", "InchesToESD"]
tank_types = ["Water", "Oil"]

# Create dictionaries for custom orders
tank_metrics_order = {metric: idx for idx, metric in enumerate(tank_metrics)}
tank_types_order = {tank_type: idx for idx, tank_type in enumerate(tank_types)}

# Define the pattern for extracting metrics
pattern = f'^(?<is_esd>ESD-)?(?<tank_type>{"|".join(tank_types)})Tank(?<tank_number>[0-9]*)(?<tank_metric>{"|".join(tank_metrics)})'

# Apply the pattern to extract and unnest the metrics
lf = lf.with_columns(separated_metrics=pl.col("metric_nice_name").str.extract_groups(pattern))
lf = lf.unnest("separated_metrics")

# Drop unwanted columns
lf = lf.drop("is_esd", "tank_number")

# Filter out unwanted rows
filtered_lf = lf.filter(~pl.col("metric_nice_name").str.ends_with("ID"))

# Convert timestamps to datetime
ts_to_sec = filtered_lf.with_columns(
    pl.col("ts").dt.strftime("%Y-%m-%d %H:%M:%S").str.strptime(pl.Datetime).alias("ts")
)

# Aggregate the data
aggregated_lf = ts_to_sec.group_by(["tank_metric", "tank_type", "uom"]).agg([
    pl.col("ts").alias("timestamps"),
    pl.col("value").alias("values")
])

# Add custom order columns
aggregated_lf = aggregated_lf.with_columns(
    pl.col("tank_metric").replace(tank_types_order, default=-1).alias("type_order")
)

aggregated_lf = aggregated_lf.drop("type_order")

# Collect the results
result_df = aggregated_lf.collect()

result_df


tank_metric,tank_type,uom,timestamps,values
str,str,str,list[datetime[μs]],list[f64]
"""InchesUntilAlarm""","""Water""","""in""","[2024-08-29 16:45:36, 2024-08-26 00:32:42, … 2024-08-26 07:06:14]","[266.361128, 243.027024, … 236.480047]"
"""Level""","""Oil""","""in""","[2024-08-25 06:44:07, 2024-08-24 19:16:58, … 2024-08-22 21:58:48]","[78.128182, 73.656298, … 78.495096]"
"""Volume""","""Oil""","""bbl""","[2024-08-22 20:10:28, 2024-08-26 18:34:13, … 2024-08-26 06:08:41]","[291.456951, 298.883254, … 308.161086]"
"""InchesUntilAlarm""","""Oil""","""in""","[2024-08-23 16:26:59, 2024-08-27 02:03:33, … 2024-08-29 07:27:55]","[268.078822, 235.166886, … 272.102104]"


In [ ]:
tank_metrics = ["Level", "Volume", "InchesUntilAlarm", "InchesToESD"]
tank_types = ["Water", "Oil"]

tank_metrics_str= "|".join(tank_metrics)
tank_types_str = "|".join(tank_types)

pattern = f'^(?<is_esd>ESD-)?(?<tank_type>{tank_types_str})Tank(?<tank_number>[0-9]*)(?<tank_metric>{tank_metrics_str})'
lf = lf.with_columns(separated_metrics = pl.col("metric_nice_name").str.extract_groups(pattern))
lf = lf.unnest("separated_metrics")
lf.collect()

In [ ]:
lf = lf.drop("is_esd", "tank_number")
lf.collect()

In [ ]:
filtered_lf = lf.filter(~pl.col("metric_nice_name").str.ends_with("ID"))
filtered_lf.collect()

In [ ]:
ts_to_sec = filtered_lf.with_columns(
    pl.col("ts").dt.strftime("%Y-%m-%d %H:%M:%S").str.strptime(pl.Datetime).alias("ts")
)

ts_to_sec.collect()

In [ ]:
aggregated_lf = ts_to_sec.group_by(["tank_metric", "tank_type","uom"]).agg([
    pl.col("ts").alias("timestamps"),
    pl.col("value").alias("values")
])

aggregated_lf = aggregated_lf.sort("tank_metric")

aggregated_lf.collect()